# YELP! APPENDEX

> Team B1: Austin Adair, Joshua Ferris, Molly Izenson, and Brandon Pugh


## Create Mongodb

[PyMongo Documentation]()

## Load in Packages

In [11]:
import json
import pandas as pd
from pymongo import MongoClient

## Create the connection to mongodb

In [12]:
client = MongoClient('localhost', 27017)

db = client.ferrisj2
db.authenticate('ferrisj2','bigdata')

businesses_collection = db.businesses
reviews_collection = db.reviews
tips_collection = db.tips

## Upload the businesses

In [10]:
businesses = pd.read_json("businesses.json", lines=True)

ValueError: Expected object or value

In [7]:
businesses_collection.insert_many(businesses.to_dict('records'))

NameError: name 'businesses' is not defined

In [23]:
businesses_collection.count_documents({})

153499

In [13]:
businesses_collection.find_one({"business_id": "f9NumwFMBDn751xgFiRbNA"})

{'_id': ObjectId('5faf5179ff55def13d9fa68a'),
 'address': '10913 Bailey Rd',
 'business_id': 'f9NumwFMBDn751xgFiRbNA',
 'checkins': 38,
 'city': 'Cornelius',
 'is_open': 1,
 'latitude': 35.4627242,
 'longitude': -80.8526119,
 'name': 'The Range At Lake Norman',
 'zip': 28031,
 'review_count': 36,
 'stars': 3.5,
 'state': 'NC',
 'zbp_employees': 8164,
 'zbp_establishments': 907,
 'zbp_annual_payroll': 284254}

## Upload the Reviews

In [15]:
reviews = pd.read_csv("reviews.csv")
reviews_collection.insert_many(reviews.to_dict('records'))

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
reviews_collection.count_documents({})

8021124

In [19]:
reviews_collection.find_one({"review_id": "6TdNDKywdbjoTkizeMce8A"})

{'_id': ObjectId('5faf52fdff55def13da1fe2a'),
 'business_id': 'IS4cv902ykd8wj1TR0N3-A',
 'cool': 0,
 'date': '2017-01-14 21:56:57',
 'funny': 0,
 'review_id': '6TdNDKywdbjoTkizeMce8A',
 'stars': 4.0,
 'text': 'Oh happy day, finally have a Canes near my casa. Yes just as others are griping about the Drive thru is packed just like most of the other canes in the area but I like to go sit down to enjoy my chicken. The cashiers are pleasant and as far as food wise i have yet to receive any funky chicken. The clean up crew zips around the dining area constantly so it\'s usually well kept. My only gripe is the one fella with Red hair he makes the rounds while cleaning but no smile or personality a few nights ago he tossed the napkins i just put on the table to help go with my meal. After I was done he just reached for my tray no "excuse me or are you done with that?"  I realize he\'s trying to do his job quickly but a little table manners goes along way. That being said still like to grub her

# Yelp Dataset Exploration



## Packages

In [13]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import urllib
import numpy as np
import matplotlib.pyplot as plt

## Load in the data

In [14]:
businesses = []
for line in open('yelp_academic_dataset_business.json', 'r'):
    businesses.append(json.loads(line))
len(businesses)

209393

In [15]:
checkins = []
for line in open('yelp_academic_dataset_checkin.json', 'r'):
    checkins.append(json.loads(line))
len(checkins)

175187

# Add Checkins to businesses

In [16]:
checkins_dict = {}
for checkin in checkins:
    cins = checkin["date"].split(',')
    checkins_dict[checkin["business_id"]] = len(cins)
for i in range(0, len(businesses)):
    businesses[i]["checkins"] = checkins_dict.get(businesses[i]["business_id"], 0)

## Fix Columns and Reduce to businesses in the US

> Specificially AZ, IL, NC, NV, OH, PA, SC, WI

In [17]:
df = json_normalize(businesses)
df["categories"] = df["categories"].fillna("Unknown")
df.drop(list(df.filter(regex = 'attributes|hours')), axis="columns", inplace=True)
df.drop(["categories"], axis="columns", inplace=True)
df.rename(columns = {'postal_code':'zip'}, inplace = True)
df = df[df.zip.str.isdigit()]
acceptable_states = ["AZ", "IL", "NC", "NV", "OH", "PA", "SC", "WI"]
df = df[df.state.isin(acceptable_states)]
df.shape

(153499, 12)

# Fetch 2018 ZBP Census data by zip code

> ZIP Code Business Patterns (ZBP) is an annual series that provides economic data by ZIP Code. This table includes the number of establishments, employment during the week of March 12, first quarter payroll, and annual payroll for All Industries (NAICS 00) by 5-digit ZIP Code.

https://api.census.gov/data/2018/zbp.html

In [18]:
zips = list(set(df["zip"]))
baseurl = "https://api.census.gov/data/2010/zbp?"
q = {"get":'EMP,ESTAB,PAYANN', "for":"zipcode:"+','.join(zips),}
fullurl = baseurl + urllib.parse.urlencode(q)
result = urllib.request.urlopen(fullurl).read()
resd = json.loads(result)
with open('zbp_data.json', 'w') as outfile:
    json.dump(resd, outfile)

# Add Census Data to Businesses Dataframe

> First impute missing values

In [ ]:
zbp_data = pd.DataFrame(resd[1:], columns=["zbp_employees", "zbp_establishments", "zbp_annual_payroll", "zip"])
zbp_data["zbp_employees"] = pd.to_numeric(zbp_data['zbp_employees'])
zbp_data["zbp_establishments"] = pd.to_numeric(zbp_data['zbp_establishments'])
zbp_data["zbp_annual_payroll"] = pd.to_numeric(zbp_data['zbp_annual_payroll'])
zbp_data.replace(0, np.nan, inplace=True)
zbp_employees_median = zbp_data["zbp_employees"].median()
zbp_establishments_median = zbp_data["zbp_establishments"].median()
zbp_annual_payroll_median = zbp_data["zbp_annual_payroll"].median()
zbp_data.fillna(zbp_data.median(),inplace=True)

new_data = {
    "zbp_employees": [],
    "zbp_establishments": [],
    "zbp_annual_payroll": [],
}

for index, row in df.iterrows():
    cur_zbp = zbp_data[zbp_data["zip"] == row["zip"]]
    if cur_zbp.empty:
        new_data["zbp_employees"].append(zbp_employees_median)
        new_data["zbp_establishments"].append(zbp_establishments_median)
        new_data["zbp_annual_payroll"].append(zbp_annual_payroll_median)
    else:
        new_data["zbp_employees"].append(cur_zbp.iloc[0]["zbp_employees"])
        new_data["zbp_establishments"].append(cur_zbp.iloc[0]["zbp_establishments"])
        new_data["zbp_annual_payroll"].append(cur_zbp.iloc[0]["zbp_annual_payroll"])

df["zbp_employees"] = new_data["zbp_employees"]
df["zbp_establishments"] = new_data["zbp_establishments"]
df["zbp_annual_payroll"] = new_data["zbp_annual_payroll"]

In [ ]:
df.head()

# Descriptive analytics

In [ ]:
df.describe()

In [ ]:
df.describe(include = 'object')

In [ ]:
df['is_open'].value_counts()

In [ ]:
df.groupby("state")["checkins", "stars"].describe()

In [ ]:
df.hist(column="stars",bins=5,grid=False)
plt.show()

In [ ]:
df.groupby("state").count()['business_id'].plot(kind="bar")

In [ ]:
df.groupby(['state','is_open']).count()['business_id'].unstack().plot(kind="bar")

In [ ]:
df.plot(x="stars", y="checkins", kind="scatter")

In [ ]:
df.groupby(['state','is_open']).mean()['stars'].unstack().plot(kind="line")

In [ ]:
df.groupby(['state','is_open']).mean()['checkins'].unstack().plot(kind="line")

# Save the Data

In [ ]:
df.to_json(r'businesses.json')

# Review Data Cleaning

In [ ]:
import json
json_data = []

file = open('yelp_academic_dataset_review.json')
for line in file:
    json_line = json.loads(line)
    json_data.append(json_line)

print(json_data [3])

In [ ]:
import pandas as pd
import numpy as np
json_data = pd.DataFrame(json_data)

In [ ]:
print(json_data.isnull().sum())

In [ ]:
print((json_data == 0).sum())

In [ ]:
json_data.shape

In [ ]:
print((json_data['text']=="").sum())

In [ ]:
json_data['Date'] = pd.to_datetime(json_data['date']).dt.date
json_data['Time'] = pd.to_datetime(json_data['date']).dt.time

In [ ]:
json_data['cool'].nunique()

In [ ]:
json_data.drop(['useful', 'funny', 'cool'], axis=1)

In [ ]:
json_data['business_id'].nunique()

In [ ]:
json_data.to_csv ('reviews.csv', index = False, header=True)

# Sentiment Analysis

This file will iterate over all businesses located in Ohio and run sentiment analysis over the associated reviews for each business using a partially clean and fully clean corpus.

------

## Load in Packages

In [ ]:
import datetime, string, nltk
from pymongo import MongoClient
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('vader_lexicon')

## Connect to MongoDB

In [ ]:
client = MongoClient('localhost', 27017)
db = client.ferrisj2
db.authenticate('ferrisj2','bigdata')

## The analysis

In [ ]:
def review_sentiment(business_id, review_total):
    ss_neg_arr = []
    ss_neu_arr = []
    ss_pos_arr = []
    ss_compound_arr = []
    for review in db.reviews.find({'business_id': business_id}):
        current_text = review['text']
        
        tokens = word_tokenize(current_text)

        # Remove the punctuations and numbers
        tokens = [word for word in tokens if word.isalpha()]

        # Lower the tokens
        tokens = [word.lower() for word in tokens]

        # Remove stopword
        tokens = [word for word in tokens if not word in stopwords.words("english")]

        # Stem the tokens
        ps = PorterStemmer()
        tokens = [ps.stem(word) for word in tokens]

        text_clean = " ".join(tokens)
        
        sia = SentimentIntensityAnalyzer()
        ss = sia.polarity_scores(text_clean)
        
        ss_neg_arr.append(ss['neg'])
        ss_neu_arr.append(ss['neu'])
        ss_pos_arr.append(ss['pos'])
        ss_compound_arr.append(ss['compound'])

        db.reviews.update_one({
            'review_id': review['review_id']
        },
        {
            '$set': {
                'text_clean':text_clean,
                'ss_neg': ss['neg'],
                'ss_neu': ss['neu'],
                'ss_pos': ss['pos'],
                'ss_compound': ss['compound']
            }
        })
    return sum(ss_neg_arr) / review_total, sum(ss_neu_arr) / review_total, sum(ss_pos_arr) / review_total, sum(ss_compound_arr) / review_total

In [ ]:
business_cursor = db.businesses.find({'state': 'WI', 'ss_compound': {'$exists': False}}, batch_size=4, no_cursor_timeout=True)

for business in business_cursor:
    print(datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S'), ' | Business: ', business['business_id'])
    review_total = db.reviews.find({ 'business_id': business['business_id']}).count()
    print('# of reviews: ', str(review_total))
    start_time = datetime.datetime.now()

    if review_total > 0 :
        ss_neg, ss_neu, ss_pos, ss_compound = review_sentiment(business['business_id'], review_total)
        db.businesses.update_one({'business_id': business['business_id']}, {
            '$set': {'ss_neg': ss_neg, 'ss_neu': ss_neu, 'ss_pos': ss_pos, 'ss_compound': ss_compound}
        })
    else:
        db.businesses.update_one({'business_id': business['business_id']}, {
            '$set': {'ss_neg': 0, 'ss_neu': 0, 'ss_pos': 0, 'ss_compound': 0}
        })

    end_time = datetime.datetime.now()
    print('Procssing time: ', str(end_time-start_time))
    print('-'*80)

business_cursor.close()

In [ ]:
business_cursor.close()

In [ ]:
print("Todo: ", db.businesses.find({'state': 'WI', 'ss_compound': {'$exists': False}}).count())
print("Done: ", db.businesses.find({'state': 'WI', 'ss_compound': {'$exists': True}}).count())

---

## Target Variable

In [ ]:
businesses_arr = list(db.businesses.find({'state': 'WI', 'ss_compound': {'$exists': True}}))
stars = [d['stars'] for d in businesses_arr]
checkins = [d['checkins'] for d in businesses_arr]

minStars = min(stars)
maxStars = max(stars)
minCheckins = min(checkins)
maxCheckins = max(checkins)

for business in businesses_arr:
    print(datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S'), ' | Business: ', business['business_id'])
    start_time = datetime.datetime.now()

    stars_normalized = (business['stars']-minStars)/(maxStars-minStars)
    checkins_normalized = (business['checkins']-minCheckins)/(maxCheckins-minCheckins)
    target = abs(stars_normalized - checkins_normalized)
    
    print(target)
    
    db.businesses.update_one({'business_id': business['business_id']}, {
        '$set': {'target': target}
    })
    
    end_time = datetime.datetime.now()
    print('Procssing time: ', str(end_time-start_time))
    print('-'*80)

In [ ]:
db.businesses.find_one({'target': {'$exists': True}})

## Modeling

Packages

In [ ]:
import pandas as pd
from numpy import arange
from pymongo import MongoClient
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, RandomForestRegressor
from sklearn.metrics import explained_variance_score, max_error, mean_absolute_error, mean_squared_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score

## Load in Data 

In [ ]:
client = MongoClient('localhost', 27017)
db = client.ferrisj2
db.authenticate('ferrisj2','bigdata')

b_arr = list(db.businesses.find({'state': 'WI', 'target': {'$exists': True}}))

df = pd.DataFrame(b_arr)

## Training and Validation Split

In [ ]:
X = df[['review_count','zbp_employees','zbp_establishments','zbp_annual_payroll','ss_compound','ss_neg','ss_neu','ss_pos']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

---
## Linear Regression
### Original Settings

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

## Ordinary Least Squares Regression

In [ ]:
from statsmodels.api import OLS
OLS(y_train,X_train).fit().summary()

---
## Lasso Model
### Original Settings

In [ ]:
regressor = Lasso()
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

### Cross Validation

In [ ]:
tunegrid = {
    'alpha': arange(0.01, 0.12, 0.01)
}

regressor = Lasso()
grid_search = GridSearchCV(estimator=regressor, param_grid=tunegrid, cv=10, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(best_grid)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

---
## Elastic Model
### Original Settings

In [ ]:
regressor = ElasticNet(random_state=1337)
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

### Cross Validation

In [ ]:
tunegrid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0],
    'l1_ratio': arange(0, 1, 0.01),
    'max_iter': range(1000, 6000, 1000)
}

regressor = ElasticNet(random_state=1337)
grid_search = GridSearchCV(estimator=regressor, param_grid=tunegrid, cv=10, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(best_grid)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

---
## ADABoost
### Original Settings

In [ ]:
regressor = AdaBoostRegressor(random_state=1337)
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

### Cross Validation

In [ ]:
tunegrid = {
    'n_estimators': range(10, 50, 5),
    'learning_rate': arange(0.01, 0.2, 0.01),
    'loss': ['linear', 'square', 'exponential'],
}

regressor = AdaBoostRegressor(random_state=1337)
grid_search = GridSearchCV(estimator=regressor, param_grid=tunegrid, cv=10, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(best_grid)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

In [ ]:
grid_search.best_params_

---
## Gradient Boosting
### Original Settings

In [ ]:
regressor = GradientBoostingRegressor(random_state=1337)
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

### Cross Validation

In [ ]:
tunegrid = {
    'n_estimators': range(100, 500, 100),
    'learning_rate': range(0.02, 0.12, 0.02),
    'max_depth': range(1, 5),
    'loss': ['huber'],
    'alpha': arange(0.1, 0.3, 0.1),
}

regressor = GradientBoostingRegressor(random_state=1337)
grid_search = GridSearchCV(estimator=regressor, param_grid=tunegrid, cv=10, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(best_grid)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

In [ ]:
grid_search.best_params_

---
## Decision Tree
### Default Settings

In [ ]:
regressor = DecisionTreeRegressor(random_state=1337, max_depth=10, min_samples_split=5, max_leaf_nodes=12)
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

### Cross Validation

In [ ]:
param_grid = {
    'max_depth': range(3, 11),
    'min_samples_split': range(20, 120, 20),
    'max_leaf_nodes': range(4, 20, 2)
}

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=10, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(best_grid)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

In [ ]:
grid_search.best_params_

---
## Random Forest
### Default Settings

In [ ]:
regressor = RandomForestRegressor(random_state = 1337)
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

### Cross Validation

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': range(3, 11),
    'max_features': range(1,8),
    'min_samples_leaf': range(3,10),
    'min_samples_split': range(20, 120, 20),
    'n_estimators': range(300, 700, 100)
}

grid_search = GridSearchCV(estimator = regressor, param_grid = param_grid, cv = 10, n_jobs = -1, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(best_grid)
print('MSE: ', mean_squared_error(y_test, y_pred), ' | R2: ', r2_score(y_test, y_pred))

In [ ]:
grid_search.best_params_